In [8]:
%matplotlib inline

import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from utils.analyze_utils import getOveralStats

In [9]:
log_path = '../log.csv'
log_df = pd.read_csv(log_path, header=None, index_col=0)

In [5]:
# Create dataframes for each model

model_names = ['BSUV-Net-2']
model2df = {}

all_rows = ['category', 'scene', 'metric']
for model_name in model_names:
    model2df[model_name] = log_df.loc[model_name]

KeyError: 'BSUV-Net-2'

In [6]:
# Combine results into a single dataframe

comparison_df = log_df.loc[['category', 'scene', 'metric']]

for model_name, model_df in model2df.items():
    model_df = model_df.apply(pd.to_numeric, errors='coerce').replace(0, np.nan).median(skipna=True)
    model_df[0] = 0
    nan_idx = ~model_df.apply(np.isnan)
    model_df = model_df.loc[nan_idx]
    model_df.name = model_name
    comparison_df = comparison_df.append(model_df)

comparison_df

,1,2,3,4,5,6,7,8,9,10,...,150,151,152,153,154,155,156,157,158,159
0,,,,,,,,,,,,,,,,,,,,,
category,baseline,baseline,baseline,baseline,baseline,baseline,baseline,baseline,baseline,baseline,...,turbulence,turbulence,turbulence,turbulence,turbulence,turbulence,turbulence,turbulence,turbulence,turbulence
scene,highway,highway,highway,pedestrians,pedestrians,pedestrians,office,office,office,PETS2006,...,turbulence0,turbulence1,turbulence1,turbulence1,turbulence2,turbulence2,turbulence2,turbulence3,turbulence3,turbulence3
metric,FNR,Prec,F-score,FNR,Prec,F-score,FNR,Prec,F-score,FNR,...,F-score,FNR,Prec,F-score,FNR,Prec,F-score,FNR,Prec,F-score


In [7]:
# Show the results

metric = 'F-score'
group_by = 'category'

metric_locs = comparison_df.iloc[2].str.contains(metric)
metric_locs = metric_locs.fillna(True)
metric_locs[0] = True
metric_df = comparison_df.loc[:, metric_locs].transpose()
metric_df[[*model_names]] = metric_df[[*model_names]].apply(pd.to_numeric)


if metric=='FNR':
    cat_based_df = 1-metric_df.groupby([group_by]).mean()
else:
    cat_based_df = metric_df.groupby([group_by]).mean()
cat_based_df.head(20)


cat_based_df.loc['MEAN'] = cat_based_df.mean()
cat_based_df = cat_based_df.round(4)
                      
cat_based_df

KeyError: "None of [Index(['BSUV-Net-2'], dtype='object', name=0)] are in the [columns]"